In [126]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [127]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [128]:
# Generate summary statistics
df_market_data.describe()

In [129]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [130]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_market_data)

In [131]:
# Create a DataFrame with the scaled data
scaled_df = pd.DataFrame(
    scaled_data,
    columns=df_market_data.columns
)

# Copy the crypto names from the original DataFrame
scaled_df["coin_id"] = df_market_data.index

# Set the coin_id column as index
scaled_df = scaled_df.set_index("coin_id")

# Display the scaled DataFrame
scaled_df.head(10)

---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [132]:
# Create a list with the number of k-values from 1 to 11
k_list = list(range(1, 11))

In [133]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for k in k_list:
    model = KMeans(n_clusters=k, random_state=0)
    model.fit(scaled_df)
    inertia.append(model.inertia_)


In [134]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    "k": k_list,
    "inertia": inertia
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)


In [135]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
elbow_plot = df_elbow.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=k_list,
    color="orange"
)
elbow_plot


#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** k = 4

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [136]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=0)

In [137]:
# Fit the K-Means model using the scaled DataFrame
model.fit(scaled_df)

In [138]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
predictions = model.predict(scaled_df)

# Print the resulting array of cluster values.
print(predictions)

In [139]:
# Create a copy of the scaled DataFrame
clustered_df_copy = scaled_df.copy()

In [140]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
clustered_df_copy["cluster"] = model.labels_

# Display the copy of the scaled DataFrame
clustered_df_copy.head(10)

In [141]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
cluster_plot = clustered_df_copy.hvplot.scatter(
    title="Crypto Clusters",
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="cluster",
    hover_cols=["coin_id"],
    marker=["triangle", "circle", "diamond", "cross"]
)
cluster_plot


---

### Optimize Clusters with Principal Component Analysis.

In [142]:
# Create a PCA model instance and set `n_components=3`.
pca_model = PCA(n_components=3)

In [143]:
# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
pca_data = pca_model.fit_transform(scaled_df)

# View the scaled PCA data
print(pca_data)

In [144]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
explained_variance = pca_model.explained_variance_ratio_
print(explained_variance)
print(explained_variance.sum())


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 0.895031657030984, which is roughly 89.5%

In [145]:
# Create a new DataFrame with the PCA data.
pca_df = pd.DataFrame(
    pca_data,
    columns=["component 1", "component 2", "component 3"]
)

# Copy the crypto names from the original scaled DataFrame
pca_df["coin_id"] = scaled_df.index

# Set the coin_id column as index
pca_df = pca_df.set_index("coin_id")

# Display the scaled PCA DataFrame
pca_df.head(10)


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [146]:
# Create a list with the number of k-values from 1 to 11
k_list = list(range(1, 11))

In [147]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for k in k_list:
    model = KMeans(n_clusters=k, random_state=0)
    model.fit(pca_df)
    inertia.append(model.inertia_)


In [148]:
# Create a dictionary with the data to plot the Elbow curve
pca_elbow_data = {
    "k": k_list,
    "inertia": inertia
}

# Create a DataFrame with the data to plot the Elbow curve
pca_df_elbow = pd.DataFrame(pca_elbow_data)


In [149]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
pca_elbow_plot = pca_df_elbow.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=k_list,
    color="skyblue"
)
pca_elbow_plot


#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** k = 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No, 4 is the best k value for both.

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [150]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=0)

In [151]:
# Fit the K-Means model using the PCA data
model.fit(pca_df)

In [152]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
predictions = model.predict(pca_df)

# Print the resulting array of cluster values.
print(predictions)

In [153]:
# Create a copy of the scaled PCA DataFrame
clustered_pca_df_copy = pca_df.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
clustered_pca_df_copy["cluster"] = model.labels_

# Display the copy of the scaled PCA DataFrame
clustered_pca_df_copy.head(10)

In [154]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
cluster_plot_pca = clustered_pca_df_copy.hvplot.scatter(
    title = "Crypto Clusters (PCA)",
    x="component 1",
    y="component 2",
    by="cluster",
    hover_cols=["coin_id"],
    marker=["triangle", "circle", "diamond", "cross"]
)
cluster_plot_pca

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [155]:
# Composite plot to contrast the Elbow curves
(elbow_plot + pca_elbow_plot)

In [156]:
# Composite plot to contrast the clusters
(cluster_plot + cluster_plot_pca)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** The four distinct clusters still remain, so this means that the removed features have very little weight in clustering. The impact in this case is practically none.